Universidad del valle de Guatemala  
Dpto. Ciencias de la computacion  
Inteligencia Artificial  
Alberto Suriano  

Laboratorio 8
Andres Quinto - 18288  
Marlon Hernández - 15177  

[Repositorio_aqui](https://github.com/AndresQuinto5/IA_LAB08.git)

### Tasks 1 - Teoría  

1. Investigar el **algoritmo AC-3** y su relación con el algoritmo de **backtracking search**  
    El **algoritmo AC-3** es un algoritmo de consistencia de arcos utilizado en problemas de satisfacción de restricciones (CSP). Su objetivo es reducir los dominios de las variables eliminando valores que no tienen soporte, es decir, que no pueden formar parte de una solución consistente con las restricciones. Por otro lado, **el algoritmo de backtracking search** es una técnica de búsqueda que intenta construir una solución paso a paso, retrocediendo cuando encuentra que una asignación de valores no lleva a una solución válida. La relación entre ambos es que **AC-3** puede utilizarse antes del **backtracking** para preprocesar el CSP, reduciendo los dominios y, por lo tanto, el número de asignaciones a probar durante el backtracking  

    Un ejemplo de este algoritmo podria ser, tenes una caja de lápices de colores y quieres asegurarte de que puedes dibujar un arcoíris completo. Pero hay una regla: cada color solo puede ir al lado de ciertos colores. **El algoritmo AC-3** es como un amigo que revisa todos los lápices y se asegura de que cada uno tenga un vecino adecuado antes de empezar a dibujar. Así, cuando comiences a colorear, no te detendrás a mitad de camino porque todos los lápices están en el orden correcto para hacer un arcoíris perfecto.

    **referencias:**
    - [Algoritmo AC-3](https://en.wikipedia.org/wiki/AC-3_algorithm)
    - [Backtracking Algorithms](https://www.freecodecamp.org/news/backtracking-algorithms-recursive-search/)

    
2. Defina en sus propias palabras el término “Arc Consistency”  
    **En mis propias palabras, “Arc Consistency”** se refiere a un estado en el que, para cada par de variables en un CSP que comparten una restricción, cada valor de la primera variable tiene al menos un valor correspondiente en la segunda variable que satisface la restricción entre ellas. Esto asegura que no hay valores aislados que hagan imposible encontrar una solución completa al problema.

    Un ejemplo podria un juego de parejas de cartas. Cada carta tiene un número y debes encontrarle una pareja que tenga el mismo número. **“Arc Consistency”** significa que todas las cartas tienen al menos una pareja posible. Si alguna carta no tuviera pareja, no podrías ganar el juego. Entonces, antes de jugar, revisas todas las cartas para asegurarte de que cada una tiene una pareja y así sabes que el juego se puede ganar.

    **referencias:**
    - [Arc Consistency](https://en.wikipedia.org/wiki/Arc_consistency)
    - [Arc Consistency Explained](https://www.boristhebrave.com/2021/08/30/arc-consistency-explained/)

### Task 2 - CSP con Backtracking, Beam y Local Search

In [286]:
from typing import Dict, List, Optional, Tuple, Callable
from itertools import permutations
import heapq
import random
import time

def revise(assignment: Dict[str, str], variable1: str, variable2: str, constraint: Callable[[str, str], bool]) -> bool:
    revised = False
    domain1 = domains[variable1]
    domain2 = domains[variable2]
    for value1 in domain1[:]:
        has_support = False
        for value2 in domain2:
            if constraint(value1, value2):
                has_support = True
                break
        if not has_support:
            domains[variable1].remove(value1)
            revised = True
    return revised

def ac3(csp: 'CSP') -> bool:
    queue = [(variable1, variable2) for variable1 in csp.variables for variable2 in csp.variables if variable1 != variable2]
    while queue:
        variable1, variable2 = queue.pop(0)
        if revise(csp.assignment, variable1, variable2, csp.constraints[(variable1, variable2)]):
            if len(domains[variable1]) == 0:
                return False
            for variable3 in csp.variables - {variable1, variable2}:
                queue.append((variable3, variable1))
    return True

class CSP:
    def __init__(self, variables: List[str], domains: Dict[str, List[str]], constraints: Dict[Tuple[str, str], Callable[[str, str], bool]], assignment: Optional[Dict[str, str]] = None):
        self.variables = variables
        self.domains = domains
        self.constraints = constraints
        self.assignment = assignment or {}

# Variables (cursos)
variables = ['Curso1', 'Curso2', 'Curso3', 'Curso4', 'Curso5', 'Curso6', 'Curso7']

# Dominios (días posibles)
domains = {var: ['Lunes', 'Martes', 'Miércoles'] for var in variables}

# Asignación de estudiantes a cursos
student_courses = {
    'Estudiante1': ['Curso1', 'Curso6' ],
    'Estudiante2': ['Curso4', 'Curso3'],
    'Estudiante3': ['Curso7', 'Curso5'],
    'Estudiante4': ['Curso2',]
}

# Restricciones
def different_day_constraint(value1: str, value2: str) -> bool:
    return value1 != value2

def same_course_different_day_constraint(value1: str, value2: str) -> bool:
    for student, courses in student_courses.items():
        if value1 in courses and value2 in courses:
            return value1 != value2
    return True

constraints = {}
for variable1, variable2 in permutations(variables, 2):
    constraints[(variable1, variable2)] = different_day_constraint
    for student, courses in student_courses.items():
        if variable1 in courses and variable2 in courses:
            constraints[(variable1, variable2)] = same_course_different_day_constraint
            break

# constraints = [
#     different_day_constraint,
#     one_exam_per_day_constraint,
#     same_course_different_day_constraint
# ]

# Crear una instancia del problema CSP
exam_scheduling_csp = CSP(variables, domains, constraints)

# # Ejemplo de asignación válida
# valid_assignment_example = {
#     'Curso1': 'Lunes',
#     'Curso2': 'Martes',
#     'Curso3': 'Miércoles',
#     'Curso4': 'Lunes',
#     'Curso5': 'Miércoles',
#     'Curso6': 'Martes',
#     'Curso7': 'Lunes'
# }

# # Verificar si la asignación válida cumple con las restricciones
# is_valid_assignment = exam_scheduling_csp.is_consistent(valid_assignment_example)

# # Imprimir la asignación de estudiantes a cursos
# print("Asignación de estudiantes a cursos:")
# for student, courses in student_courses.items():
#     print(f"\n{student}: {', '.join(courses)}")

# # Imprimir la asignación válida y su validez
# print("\nAsignación válida de ejemplo:")
# for course, day in valid_assignment_example.items():
#     print(f"{course}: {day}")
# print(f"\n¿La asignación cumple con las restricciones? {is_valid_assignment}")


- Todos los examenes, tienen que ser en dias diferentes.
- Los estudiantes solo pueden tener UN examen por dia.
- Los estudiantes miembros de un mismo curso no tengan exámenes el mismo día.



#### Backtracking implementation

In [287]:
def backtracking_search(csp: 'CSP', assignment: Optional[Dict[str, str]] = None) -> Optional[Dict[str, str]]:
    if assignment is None:
        assignment = {}

    if len(assignment) == len(csp.variables):
        return assignment

    unassigned_variables = [v for v in csp.variables if v not in assignment]

    for variable in unassigned_variables:
        for value in domains[variable]:
            new_assignment = assignment.copy()
            new_assignment[variable] = value
            if ac3(CSP(csp.variables, csp.domains, csp.constraints, new_assignment)):
                result = backtracking_search(CSP(csp.variables, csp.domains, csp.constraints, new_assignment), new_assignment)
                if result is not None:
                    return result
        break

    return None

### Beam search implementation

In [288]:
def beam_search(csp, beam_width, assignment={}):
    if len(assignment) == len(csp.variables):
        return assignment

    unassigned_variables = [v for v in csp.variables if v not in assignment]

    beam = []
    for variable in unassigned_variables:
        for value in csp.domains[variable]:
            new_assignment = assignment.copy()
            new_assignment[variable] = value
            if ac3(CSP(csp.variables, csp.domains, csp.constraints, new_assignment)):
                conflicts = sum(1 for (var1, var2), constraint in csp.constraints.items()
                                if var1 in new_assignment and var2 in new_assignment
                                and not constraint(new_assignment[var1], new_assignment[var2]))
                heapq.heappush(beam, (conflicts, tuple(new_assignment.items())))

    beam = heapq.nsmallest(beam_width, beam)

    for _, new_assignment_tuple in beam:
        new_assignment = dict(new_assignment_tuple)
        result = beam_search(CSP(csp.variables, csp.domains, csp.constraints, new_assignment), beam_width, new_assignment)
        if result is not None:
            return result

    return None

### Local search implementation

In [289]:
def local_search(csp, max_iterations, assignment={}):
    if len(assignment) == 0:
        assignment = {variable: random.choice(csp.domains[variable]) for variable in csp.variables}

    for _ in range(max_iterations):
        if ac3(CSP(csp.variables, csp.domains, csp.constraints, assignment)):
            return assignment

        variable = random.choice(list(assignment.keys()))
        value = random.choice([v for v in csp.domains[variable] if v != assignment[variable]])
        new_assignment = assignment.copy()
        new_assignment[variable] = value

        if ac3(CSP(csp.variables, csp.domains, csp.constraints, new_assignment)):
            assignment = new_assignment

    return None


### Use case and comparison of solution using algorithms

In [290]:
from prettytable import PrettyTable

# Ejecutar los algoritmos y recopilar los resultados
algorithms = ['Backtracking Search', 'Beam Search', 'Local Search']
results = []

for algorithm in algorithms:
    start_time = time.time()

    if algorithm == 'Backtracking Search':
        solution = backtracking_search(exam_scheduling_csp)
    elif algorithm == 'Beam Search':
        beam_width = 5
        solution = beam_search(exam_scheduling_csp, beam_width)
    else:  # Local Search
        max_iterations = 1000
        solution = local_search(exam_scheduling_csp, max_iterations)

    end_time = time.time()
    execution_time = end_time - start_time

    if solution is not None:
        is_valid = ac3(CSP(exam_scheduling_csp.variables, exam_scheduling_csp.domains, exam_scheduling_csp.constraints, solution))
    else:
        is_valid = False

    results.append((algorithm, solution, execution_time, is_valid))

# Mostrar los resultados en una tabla utilizando prettytable
table = PrettyTable()
table.field_names = ['Algoritmo', 'Solución', 'Tiempo de ejecución (segundos)', '¿Solución válida?']
table.align = 'l'  # Alinear todo a la izquierda
table.float_format = '.10'  # Establecer el formato de flotante a 10 dígitos

for algorithm, solution, execution_time, is_valid in results:
    if solution is None:
        solution_str = 'No se encontró solución'
    else:
        solution_str = '\n'.join(f"{course}: {day}" for course, day in solution.items())

    table.add_row([algorithm, solution_str, execution_time, is_valid])

print(table)

+---------------------+-------------------+--------------------------------+-------------------+
| Algoritmo           | Solución          | Tiempo de ejecución (segundos) | ¿Solución válida? |
+---------------------+-------------------+--------------------------------+-------------------+
| Backtracking Search | Curso1: Lunes     | 0.0030386448                   | True              |
|                     | Curso2: Lunes     |                                |                   |
|                     | Curso3: Lunes     |                                |                   |
|                     | Curso4: Lunes     |                                |                   |
|                     | Curso5: Lunes     |                                |                   |
|                     | Curso6: Lunes     |                                |                   |
|                     | Curso7: Lunes     |                                |                   |
| Beam Search         | Curso1